In [1]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import netCDF4
import numpy as np
import xarray as xr
from cftime import num2pydate
from metpy.units import units
from siphon.catalog import TDSCatalog
from xarray.backends import NetCDF4DataStore

from data import GetGFSData
from indices import CalculateIndices
from charts import CalculateCharts

In [2]:
data = xr.open_dataset('data.nc')

In [3]:
charts = CalculateCharts(data)
showers = charts.showers_heat_humidity()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
indices = CalculateIndices(data)
li_index = indices.li()
li_index.min()

-9.663807

In [2]:
URL = 'http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/catalog.xml'
dataset = 'Latest Collection for GFS Quarter Degree Forecast'

variables = ['Relative_humidity_isobaric',
             'Temperature_isobaric',
             'u-component_of_wind_isobaric',
             'v-component_of_wind_isobaric',
             'Best_4_layer_Lifted_Index_surface',
             'Geopotential_height_isobaric',
             'Precipitable_water_entire_atmosphere_single_layer',
             'Pressure_reduced_to_MSL_msl']

gfs = GetGFSData(variables)
data = gfs.get()
data.to_netcdf(path='./data.nc')

Process done in 71.19587111473083 seconds
